In [135]:
#!pip install -U gymnasium
#!pip install stable-baselines3

In [136]:
from typing import Optional
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
from datetime import datetime

In [137]:
example_LOB = torch.tensor([
    [1772181000, 55.8, 2, 55.76, 6, 55.7, 4, 55.82, 8, 55.88, 2, 55.89, 4, 0.8, 55.78, 0],
    [1772181001, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772181002, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772181003, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772181004, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181005, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181006, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181007, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181008, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181009, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181010, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181011, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772181012, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772181013, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772181014, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181015, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181016, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181017, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181018, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181019, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181020, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181021, 55.78, 10, 55.73, 12, 55.71, 8, 55.81, 8, 55.84, 12, 55.87, 8, 0.65, 55.83, 1],
    [1772181022, 55.78, 10, 55.74, 3, 55.73, 12, 55.81, 8, 55.84, 12, 55.89, 4, 0.6, 55.82, 1],
    [1772181023, 55.76, 4, 55.75, 2, 55.74, 8, 55.81, 8, 55.82, 2, 55.84, 11, 0.82, 55.77, 0],
    [1772181024, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181025, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181026, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181027, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181028, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181029, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
    [1772181030, 55.81, 10, 55.79, 23, 55.78, 15, 55.83, 8, 55.85, 6, 55.93, 8, 0.44, 55.79, 0],
], dtype=torch.float64)

In [138]:
example_LOB.shape

torch.Size([31, 16])

In [139]:
def embed_positions(x, time_scale):
    sin_pos = np.sin(2 * np.pi * x / time_scale)
    cos_pos = np.cos(2 * np.pi * x / time_scale)
    return sin_pos, cos_pos

In [ ]:
fee_pct = 0.0005

class LOBEnv(gym.Env):
    def __init__(self, orderbook, device, book_depth):
        # large_cpu_tensor should be pinned for speed
        if device == 'cuda':
            self.data = orderbook.pin_memory() 
        else:
            self.data = orderbook
        self.device = device
        self.book_depth = book_depth
        self.current_step = 0
        self.max_steps = self.data.shape[0] - 1
        self.initial_cash = 100_000.0
        
        self.action_space = spaces.Discrete(3)
        self.observation_space = spaces.Box(low=-float('inf'), high=float('inf'), shape=(33,), dtype=float)

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.date = None
        self.market_open_time = None
        self.weekday = None
        self.start_year = 2022 # change this to whatever year testing starts at
        
        self.cash = self.initial_cash
        self.total_balance = self.initial_cash
        self.inventory = 0
        self.entry_price = 0
        self.steps_held = 0
        self.transaction_costs = 0
        self.total_return = 0

        self.volume_mismatches = 0
        
        #return self.data[self.current_step].to(self.device, non_blocking=True), {}
        return self._get_obs(), {}
        
    def _get_obs(self):
        # Slice the CPU tensor and move ONLY this tiny piece to GPU
        # This transfer is very fast for a single snapshot
        #LOB = self.data[self.current_step].to(self.device, non_blocking=True)
        LOB = self.data[self.current_step]
        
        current_date = datetime.fromtimestamp(LOB[0].item())
        year, hour, minute, second = current_date.year, current_date.hour, current_date.minute, current_date.second
        if hour == 9 and minute == 30 and second == 0:
            self.date = current_date
            self.market_open_time = LOB[0]
            self.month = self.date.month
            self.weekday = current_date.weekday()
        
        time_since_open = LOB[0].item() - self.market_open_time

        bid_prices = LOB[1:(self.book_depth*2)+1:2]
        bid_volumes = LOB[2:(self.book_depth*2)+1:2]

        ask_prices = LOB[(self.book_depth*2)+1:(self.book_depth*4)+1:2]
        ask_volumes = LOB[(self.book_depth*2)+2:(self.book_depth*4)+1:2]

        #print(f'bid_prices: {bid_prices}')
        #print(f'bid_volumes: {bid_volumes}')

        #print(f'ask_prices: {ask_prices}')
        #print(f'ask_volumes: {ask_volumes}')

        cumu_bid_volume = sum(bid_volumes)
        cumu_ask_volume = sum(ask_volumes)

        # dollar amount of bid and asks. how much money can be traded
        bid_liquidity = sum(bid_prices * bid_volumes)
        ask_liquidity = sum(ask_prices * ask_volumes)

        order_book_imbalance = (cumu_bid_volume - cumu_ask_volume) / (cumu_bid_volume + cumu_ask_volume)
        spread = LOB[7] - LOB[1]
        midpoint = (LOB[7] + LOB[1]) / 2
        microprice = (bid_liquidity + ask_liquidity) / (cumu_bid_volume + cumu_ask_volume)

        # create positonal embeddings
        # 60 * 60 * 24 = total seconds in a day
        daily_pos_sin, daily_pos_cos = embed_positions(time_since_open.item(), 60*60*24)
        weekday_pos_sin, weekday_pos_cos = embed_positions(current_date.weekday(), 7)
        monthly_pos_sin, monthly_pos_cos = embed_positions(self.month, 12)
        year_pos = year - self.start_year

        # create tensors that will be used for the observation space
        # create tensor private state from inventory and balance
        private_state = torch.tensor([
            self.cash / self.initial_cash,
            self.inventory,
            self.total_balance / self.initial_cash
            
        ], dtype=torch.float32)

        # create tensor for engineed features
        engineered_features = torch.stack([
            cumu_bid_volume, cumu_ask_volume, bid_liquidity, ask_liquidity, order_book_imbalance, spread, midpoint, microprice
        ])

        # create tensor for positional embeddings
        positional_embeddings = torch.tensor([
            daily_pos_sin, daily_pos_cos, weekday_pos_sin, weekday_pos_cos, monthly_pos_sin, monthly_pos_cos, year_pos
        ])

        # remove timestamp from LOB since we have positional embeddings
        LOB = LOB[1:]

        # combine all tensors together
        full_obs = torch.cat([LOB, engineered_features, private_state, positional_embeddings])
        if self.device == 'cuda':
            full_obs = full_obs.to(device=self.device, dtype=torch.float32, non_blocking=True)
        else:
            full_obs = full_obs.to(device=self.device, dtype=torch.float32)
        # print(f'observation space shape: {full_obs.shape}')
        return full_obs

    def step(self, action):
        # 1. get LOB data at current state and make necessary variables for the action taken
        current_obs = self.data[self.current_step]

        bid_prices = current_obs[0:self.book_depth*2:2]
        bid_volumes = current_obs[1:self.book_depth*2:2]
        ask_prices = current_obs[self.book_depth*2:self.book_depth*4:2]
        ask_volumes = current_obs[(self.book_depth*2)+1:self.book_depth*4:2]

        exec_shares = 0
        exec_cost = 0
        exec_fees = 0

        if action == 1: # buy order
            trade_budget = self.cash * 0.01 # 1% of portfolio value
            remaining_budget = trade_budget
            
            for i in range(len(ask_prices)):
                price = ask_prices[i].item()
                volume = ask_volumes[i].item()

                # get the maximum number of shares that can be traded based on budget
                max_shares = remaining_budget / (price * (1 + fee_pct))

                # take the minimum volume between what is available and wehat we can trade
                shares_taken = min(volume, max_shares)

                # compute metrics
                cost = shares_taken * price
                fees = cost * fee_pct
                exec_shares += shares_taken
                exec_cost += (cost + fees)
                exec_fees += fees
                remaining_budget -= (cost + fees)

                if remaining_budget <= 0 or shares_taken < volume:
                    break

            if exec_shares > 0:
                total_val = (self.inventory * self.entry_price) + exec_cost
                self.inventory += exec_shares
                self.entry_price = total_val / self.inventory
                self.cash -= exec_cost
                self.transaction_costs += exec_fees
            
        elif action == 2: # sell order
            trade_vol = self.inventory
            remaining_inventory = trade_vol

            for i in range(len(bid_prices)):
                price = bid_prices[i].item()
                volume = bid_volumes[i].item()

                # get the max number of shares that can be sold
                shares_sold = min(volume, remaining_inventory)

                # compute metrics
                cost = shares_sold * price
                fees = cost * fee_pct
                exec_shares += shares_sold
                exec_cost += (cost - fees)
                exec_fees += fees
                remaining_inventory -= shares_sold

                if remaining_inventory <= 0 or shares_sold < volume:
                    break
            # handle edge case where not all shares are sold. give warning and execute remaining sale at the worst bid price
            if exec_shares > 0:
                self.total_return += exec_cost - (self.entry_price * self.inventory)
                self.inventory = 0
                self.entry_price = 0
                self.cash += exec_cost
                self.transaction_costs += exec_fees
                
        # 2. Increment step
        self.current_step += 1

        if self.current_step >= self.max_steps:
            terminated = True
            truncated = False
        elif self.total_balance <= 0:
            terminated = False
            truncated = True 
        else:
            terminated = False
            truncated = False
        obs = self._get_obs()

        # replace these variables with the variables from the observation space
        midpoint = (obs[6].item() + obs[0].item()) / 2
        new_total_balance = self.cash + (self.inventory * midpoint)
        reward = new_total_balance - self.total_balance
        self.total_balance = new_total_balance

        return obs, reward, terminated, truncated, {}

In [203]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
torch.set_printoptions(precision=6, sci_mode=False)
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
env = LOBEnv(example_LOB, device, 3)

obs, info = env.reset()
print(obs)

for i in range(env.max_steps + 1):
    action = 1
    obs, reward, terminated, truncated, info = env.step(0)
    print(obs)

    if terminated or truncated:
        break

tensor([ 55.799999,   2.000000,  55.759998,   6.000000,  55.700001,   4.000000,
         55.820000,   8.000000,  55.880001,   2.000000,  55.889999,   4.000000,
          0.800000,  55.779999,   0.000000,  12.000000,  14.000000, 668.960022,
        781.880005,  -0.076923,   0.020000,  55.810001,  55.801537,   1.000000,
          0.000000,   1.000000,   0.000000,   1.000000,  -0.433884,  -0.900969,
          0.866025,   0.500000,   4.000000], device='mps:0')
tensor([  55.779999,   10.000000,   55.730000,   12.000000,   55.709999,
           8.000000,   55.810001,    8.000000,   55.840000,   12.000000,
          55.869999,    8.000000,    0.650000,   55.830002,    1.000000,
          30.000000,   28.000000, 1672.239990, 1563.520020,    0.034483,
           0.030000,   55.794998,   55.788967,    1.000000,    0.000000,
           1.000000,    0.000073,    1.000000,   -0.433884,   -0.900969,
           0.866025,    0.500000,    4.000000], device='mps:0')
tensor([  55.779999,   10.000000,   5

In [204]:
env.action_space

Discrete(3)

In [205]:
#LOB = example_LOB[0]
#[LOB[i] for i in range(0, 6, 2)]
book_depth = 3
bid_prices = torch.tensor([example_LOB[0][i] for i in range(0, book_depth*2, 2)])
bid_volumes = torch.tensor([example_LOB[0][i] for i in range(1, book_depth*2, 2)])

ask_prices = torch.tensor([example_LOB[0][i] for i in range(book_depth*2, book_depth*4, 2)])
ask_volumes = torch.tensor([example_LOB[0][i] for i in range((book_depth*2)+1, book_depth*4, 2)])

print(bid_prices, bid_volumes)
print(sum(bid_volumes).item())
#print(sum(bid_prices * bid_volumes).item())

print(ask_prices, ask_volumes)
print(sum(ask_volumes).item())

cumu_bid_volume = sum(bid_volumes)
cumu_ask_volume = sum(ask_volumes)
orderbook_imbalance = (cumu_bid_volume - cumu_ask_volume) / (cumu_bid_volume + cumu_ask_volume)

print(orderbook_imbalance)

spread = example_LOB[0][6] - example_LOB[0][0]
print(spread)

bid_liquidity = sum(bid_prices * bid_volumes)
ask_liquidity = sum(ask_prices * ask_volumes)

print(bid_liquidity)
print(ask_liquidity)

tensor([1772181000.,          2.,          6.], dtype=torch.float64) tensor([55.800000, 55.760000, 55.700000], dtype=torch.float64)
167.26
tensor([4., 8., 2.], dtype=torch.float64) tensor([55.820000, 55.880000, 55.890000], dtype=torch.float64)
167.59
tensor(-0.000986, dtype=torch.float64)
tensor(-1772180996., dtype=torch.float64)
tensor(98887700245.720001, dtype=torch.float64)
tensor(782.100000, dtype=torch.float64)


In [206]:
max_trade_amount = 100_000 * 0.01 # 1% of portfolio value
max_trade_amount

1000.0

In [207]:
print(example_LOB[0][0:6:2])
print(example_LOB[0][1:6:2])

tensor([1772181000.,          2.,          6.], dtype=torch.float64)
tensor([55.800000, 55.760000, 55.700000], dtype=torch.float64)


In [208]:
bid_prices = example_LOB[0][book_depth*2:book_depth*4:2]
bid_volumes = example_LOB[0][(book_depth*2)+1:book_depth*4:2]

In [209]:
print(bid_prices)
print(bid_volumes)

tensor([4., 8., 2.], dtype=torch.float64)
tensor([55.820000, 55.880000, 55.890000], dtype=torch.float64)


In [210]:
torch.tensor([1,])

tensor([1])

In [211]:
microprice = (bid_liquidity + ask_liquidity) / (cumu_bid_volume + cumu_ask_volume)
microprice

tensor(295319399.814305, dtype=torch.float64)